# Tutorial: MongoDB Search Queries

...run this in a notebook on the server...

## Load .nii for Muenster-VBM HC-MDD Data

Do some importing...

In [4]:
import pymongo
from pymongo import MongoClient
import pprint
import time
import os
import numpy as np
from bson.objectid import ObjectId
import pandas as pd

Now we need to connect to the MongoDB, grab the database (photondb) and grab the collection (which is called data in our case)...

In [5]:
client = MongoClient('localhost', 27017)
photondb = client.photon
data = photondb.data

Now we want all subjects for which a VBM file exists and who are either healthy controls (HC) or have a major depressive disorder (MDD)...
1. We need to iterate over a subcollection of the data.
2. We get this subcollection by using the .find() method.
3. In the .find() method, we specify our search query which consists of an AND and an OR statement. (This is pymongo and mongodb syntax. You should get used to it...)
4. If we like to retrieve the data, we simply use "sub" which is a nested dictionary containing all information of this document (one subject). Since it is a dictionary, we use standard python syntax to retrieve relevant fields.

In [30]:
for sub in data.find({ '$and': [
    { 'session.0.images.VBM': {"$exists": True}},
    {'$or': [
        {'session.0.disorder': 'HC'},
        {'session.0.disorder': 'MDD'}]}]}):
    
    print('Subject:', sub['ID'], '\nDiagnosis:', sub['session'][0]['disorder'], 
          '\nPath:', sub['session'][0]['images']['VBM']['path'],
          '\nImage:',  sub['session'][0]['images']['VBM']['filename'],'\n')

Subject: F0001 
Diagnosis: HC 
Path: /data/vbm_bipolar/AllStudies_GM_VBM8_renamed/AllStudies_GM_VBM8_renamed/ 
Image: s8m0wrp1F0001_t1mprsagp2iso20140911PSYCHMACS0001A1s003a1001.nii 

Subject: F0008 
Diagnosis: HC 
Path: /data/vbm_bipolar/AllStudies_GM_VBM8_renamed/AllStudies_GM_VBM8_renamed/ 
Image: s8m0wrp1F0008_t1mprsagp2iso20140926PSYCHMACS0008C1s003a1001.nii 

Subject: F0009 
Diagnosis: HC 
Path: /data/vbm_bipolar/AllStudies_GM_VBM8_renamed/AllStudies_GM_VBM8_renamed/ 
Image: s8m0wrp1F0009_t1mprsagp2iso20140926PSYCHMACS0009A1s003a1001.nii 

Subject: F0014 
Diagnosis: HC 
Path: /data/vbm_bipolar/AllStudies_GM_VBM8_renamed/AllStudies_GM_VBM8_renamed/ 
Image: s8m0wrp1F0014_t1mprsagp2iso20141016PSYCHMACS0014C1s003a1001.nii 

Subject: F0017 
Diagnosis: HC 
Path: /data/vbm_bipolar/AllStudies_GM_VBM8_renamed/AllStudies_GM_VBM8_renamed/ 
Image: s8m0wrp1F0017_t1mprsagp2iso20141017PsychMACS0017A1s003a1001.nii 

Subject: F0019 
Diagnosis: HC 
Path: /data/vbm_bipolar/AllStudies_GM_VBM8_rename

##### So how many subjects do we have here?

In [9]:
data.count({ '$and': [
    { 'session.0.images.VBM': {"$exists": True}},
    {'$or': [
        {'session.0.disorder': 'HC'},
        {'session.0.disorder': 'MDD'}]}]})

2752

##### Now get the data and store it in a pandas dataframe

In [19]:
ids = []
group = []
images = []
scanner = []
studyId = []
for sub in data.find({ '$and': [
    { 'session.0.images.VBM': {"$exists": True}},
    {'$or': [
        {'session.0.disorder': 'HC'},
        {'session.0.disorder': 'MDD'}]}]}):
    
    ids.append(sub['ID'])
    group.append(sub['session'][0]['disorder'])
    image = '/home/nils/data/' + sub['session'][0]['images']['VBM']['path'] + sub['session'][0]['images']['VBM']['filename']
    images.append(image)
    scanner.append(sub['session'][0]['images']['VBM']['scanner'])
    studyId.append(sub['StudyID'])

df = pd.DataFrame([ids, group, studyId, scanner, images]).transpose()

So there we go. All we need to do now is to load the niftis and we're done...

In [26]:
df

,0,1,2,3,4
0,F0001,HC,FOR2107-marburg,siemens-marburg,/home/nils/data//data/vbm_bipolar/AllStudies_G...
1,F0008,HC,FOR2107-marburg,siemens-marburg,/home/nils/data//data/vbm_bipolar/AllStudies_G...
2,F0009,HC,FOR2107-marburg,siemens-marburg,/home/nils/data//data/vbm_bipolar/AllStudies_G...
3,F0014,HC,FOR2107-marburg,siemens-marburg,/home/nils/data//data/vbm_bipolar/AllStudies_G...
4,F0017,HC,FOR2107-marburg,siemens-marburg,/home/nils/data//data/vbm_bipolar/AllStudies_G...
5,F0019,HC,FOR2107-marburg,siemens-marburg,/home/nils/data//data/vbm_bipolar/AllStudies_G...
6,F0021,HC,FOR2107-marburg,siemens-marburg,/home/nils/data//data/vbm_bipolar/AllStudies_G...
7,F0022,HC,FOR2107-marburg,siemens-marburg,/home/nils/data//data/vbm_bipolar/AllStudies_G...
8,F0023,HC,FOR2107-marburg,siemens-marburg,/home/nils/data//data/vbm_bipolar/AllStudies_G...
9,F0024,HC,FOR2107-marburg,siemens-marburg,/home/nils/data//data/vbm_bipolar/AllStudies_G...


### Subjects of only one study

To get only subjects of a specific study, we add another field to the "$and" operator in our search query.

In [37]:
ids = []
group = []
images = []
scanner = []
studyId = []
for sub in data.find({ '$and': [{ 'session.0.images.VBM': {"$exists": True}},
                                {'StudyID': 'FOR2107-muenster'},
                                {'$or': [
                                    {'session.0.disorder': 'HC'},
                                    {'session.0.disorder': 'MDD'}]}
                                ]}):
    
    ids.append(sub['ID'])
    group.append(sub['session'][0]['disorder'])
    image = '/home/nils/data/' + sub['session'][0]['images']['VBM']['path'] + sub['session'][0]['images']['VBM']['filename']
    images.append(image)
    scanner.append(sub['session'][0]['images']['VBM']['scanner'])
    studyId.append(sub['StudyID'])

df = pd.DataFrame([ids, group, studyId, scanner, images]).transpose()

In [38]:
df

,0,1,2,3,4
0,F2004,HC,FOR2107-muenster,siemens-muenster,/home/nils/data//data/vbm_bipolar/AllStudies_G...
1,F2005,HC,FOR2107-muenster,siemens-muenster,/home/nils/data//data/vbm_bipolar/AllStudies_G...
2,F2007,HC,FOR2107-muenster,siemens-muenster,/home/nils/data//data/vbm_bipolar/AllStudies_G...
3,F2008,HC,FOR2107-muenster,siemens-muenster,/home/nils/data//data/vbm_bipolar/AllStudies_G...
4,F2009,HC,FOR2107-muenster,siemens-muenster,/home/nils/data//data/vbm_bipolar/AllStudies_G...
5,F2010,HC,FOR2107-muenster,siemens-muenster,/home/nils/data//data/vbm_bipolar/AllStudies_G...
6,F2011,HC,FOR2107-muenster,siemens-muenster,/home/nils/data//data/vbm_bipolar/AllStudies_G...
7,F2013,HC,FOR2107-muenster,siemens-muenster,/home/nils/data//data/vbm_bipolar/AllStudies_G...
8,F2014,HC,FOR2107-muenster,siemens-muenster,/home/nils/data//data/vbm_bipolar/AllStudies_G...
9,F2015,HC,FOR2107-muenster,siemens-muenster,/home/nils/data//data/vbm_bipolar/AllStudies_G...


### Filter for Scanner

In [49]:
ids = []
group = []
images = []
scanner = []
studyId = []
for sub in data.find({ '$and': [{ 'session.0.images.VBM': {"$exists": True}},
                                {'$or': [
                                    {'session.0.images.VBM.scanner': 'philips-intera-muenster'},
                                    {'session.0.images.VBM.scanner': 'philips-intera'}]},
                                {'$or': [
                                    {'session.0.disorder': 'HC'},
                                    {'session.0.disorder': 'MDD'}]}
                                ]}):
    
    ids.append(sub['ID'])
    group.append(sub['session'][0]['disorder'])
    image = '/home/nils/data/' + sub['session'][0]['images']['VBM']['path'] + sub['session'][0]['images']['VBM']['filename']
    images.append(image)
    scanner.append(sub['session'][0]['images']['VBM']['scanner'])
    studyId.append(sub['StudyID'])

df = pd.DataFrame([ids, group, studyId, scanner, images]).transpose()

In [50]:
df

,0,1,2,3,4
0,H0001,HC,bidirect,philips-intera,/home/nils/data//data/vbm_bipolar/AllStudies_G...
1,H0003,HC,bidirect,philips-intera,/home/nils/data//data/vbm_bipolar/AllStudies_G...
2,H0004,HC,bidirect,philips-intera,/home/nils/data//data/vbm_bipolar/AllStudies_G...
3,H0005,HC,bidirect,philips-intera,/home/nils/data//data/vbm_bipolar/AllStudies_G...
4,H0006,HC,bidirect,philips-intera,/home/nils/data//data/vbm_bipolar/AllStudies_G...
5,H0007,HC,bidirect,philips-intera,/home/nils/data//data/vbm_bipolar/AllStudies_G...
6,H0008,HC,bidirect,philips-intera,/home/nils/data//data/vbm_bipolar/AllStudies_G...
7,H0009,HC,bidirect,philips-intera,/home/nils/data//data/vbm_bipolar/AllStudies_G...
8,H0011,HC,bidirect,philips-intera,/home/nils/data//data/vbm_bipolar/AllStudies_G...
9,H0012,HC,bidirect,philips-intera,/home/nils/data//data/vbm_bipolar/AllStudies_G...
